## Métodos do Pandas:
- criação DataFrame
- sort, drop, index, rename
- append, merge e concat
- group by
- crosstab e pivot_table <https://pbpython.com/pandas-pivot-table-explained.html>
- multiindices
- total linha, coluna, DataFrame

In [1]:
import pandas as pd
import numpy as np

In [ ]:
d_configuracao = {'display.max_columns': 1000,
                  'display.expand_frame_repr': True,
                  'display.max_rows': 10,
                  'display.precision': 2,
                  'display.show_dimensions': True,
                 'display.float_format': '{:,.2f}'.format}

for op, value in d_configuracao.items():
    pd.set_option(op, value)
    print(op, value)

In [ ]:
pd.options.display.float_format = 'R${:,.2f}'.format
pd.get_option("display.max_rows")
pd.set_option("display.max_rows", 101)
pd.get_option("display.max_columns")
pd.set_option("display.max_columns", 101)

In [ ]:
pd.read_excel('nome_arq.xlsx', sheet_name = 'nome_plan')

In [ ]:
import chardet
with open('pedidos.csv', 'rb') as f:
    result = chardet.detect(f.read())
print(result)
pd.read_csv('nome_arq.csv', encoding=result[0], sep=';')

In [ ]:
df.describe(exclude='number')   # include

In [ ]:
gp = df.groupby('cat1', 'cat2').vel.agg(['sum','count']).sort_values(by=['count','sum'],ascending=False)
df.groupby(['cat1','cat2'])[['quant1', 'quant2']].agg([np.sum, np.mean])
gp['percentual'] = gp.apply(lambda x: x/x.sum())

In [ ]:
gp['perc_cat1'] = gp.groupby(level=0).apply(lambda x: x/x.sum())  # % por cat1

In [ ]:
gp.cumsum()
gp.groupby(level=0).cumsum()

In [ ]:
df.groupby(['cat1', 'cat2']).agg({'quant1':[np.sum, np.mean], 'quant2':['sum']}).sort_values(by=[('imposto', 'sum')], ascending=False).head(6)

In [ ]:
gp = df.groupby('cat1').agg({'num1': [('sum_num1', 'sum'), ('mean_num1', 'mean')]})
gp.columns = gp.columns.droplevel()

In [ ]:
# pivot sem agregar os dados (mantendo o índice):
df.pivot(columns="col1", values="col2")

In [ ]:
df.pivot_table(index= ['col1', 'col2'],
              aggfunc=[np.sum], fill_value=0, columns= ['col3'], 
               values= ['col_vlr4', 'col_qtde_5'])

In [ ]:
pd.concat([df1, df2])  # axis=0, join= 'outer'    empilhamento vertical

In [ ]:
pd.concat([df1, df2], axis= 1)
pd.merge(df1, df2, how='left',left_index=True, right_index=True)

In [ ]:
pd.merge(df1, df2, how= 'left', on= 'cnae')  # cuidado com o índice original

In [ ]:
df1.append([df2, df3])

In [ ]:
# lag
df['col_lag1'] = df.col.shift(1)
df['col_lag2'] = df.col.shift(2)

In [ ]:
# col no formato aaaa-mm-dd
df.index = pd.to_datetime(df.mes, format='%Y-%m-%d')
df['ano'] = df.index.to_period('A')
df['ano_mes'] = df.index.to_period('M')

In [ ]:
# dia para mes ou ano
df.resample('M').mean()

In [2]:
nf = pd.DataFrame(data= np.array([[201904, 1], 
                                  [201905, 1], 
                                  [201905, 2], 
                                  [201906, 2],
                                 [201906, 3]])
             , columns= ['dt', 'prod'])
aliq = pd.DataFrame(data= np.array([[1, 201801, 201904, 0.4], 
                                    [1, 201905, 201906, 0.5], 
                                    [2, 201903, 201905, 0.7],
                                   [2, 201906, 201912, 0.8],
                                   [2, 202001, 202012, 0.92]]),
                   columns= ['prod', 'dt_ini', 'dt_fim', 'aliq'])

In [3]:
# atribui aliq a nf
pd.merge(nf, aliq, how= 'left', on=['prod']).query('dt >= dt_ini & dt <= dt_fim')#.drop(columns=['dt_ini', 'dt_fim'])

,dt,prod,dt_ini,dt_fim,aliq
0,201904,1,201801.0,201904.0,0.4
3,201905,1,201905.0,201906.0,0.5
4,201905,2,201903.0,201905.0,0.7
8,201906,2,201906.0,201912.0,0.8


In [4]:
# registros nf ausentes da tab de aliquotas
pd.merge(nf, aliq, how= 'left', on=['prod']).pipe(lambda df: df.loc[df.aliq.isna()])

,dt,prod,dt_ini,dt_fim,aliq
10,201906,3,NaN,NaN,NaN


In [ ]:
df.iloc[:,0:5].astype('category')

In [ ]:
df[['col1','col2']].max(axis=1)

In [3]:
# apply a function to DataFrame:
df = pd.DataFrame(data= np.array([[10, 1],
                                 [20, 2]]), columns = ['c0', 'c1'])
def imput(cols):
    return cols[0] + cols[1] # soma na linha

df['c2'] = df[['c0', 'c1']].apply(imput, axis= 1)
df

,c0,c1,c2
0,10,1,11
1,20,2,22


In [7]:
df.c2.apply(lambda x: x*x if (x < 20) else x+x)

0    121
1     44
Name: c2, dtype: int64

In [ ]:
bins= [0.3, 0.5, 1, 2.5, 5, 10, 1500]
col['intervalo'] = pd.cut(df.col_valor, bins=bins)

In [ ]:
# registros nulos
sns.heatmap(df.isna(), yticklabels= False, cbar= False, cmap= 'viridis')
df.isna().sum().sort_values(ascending= False)
# drop nulos
df.dropna(axis=1, inplace=True)  # subset=[col1, col2] col com nulos; axis=0=defalut= apaga linhas; 1=colunas
# tratamento: verficando col correlacionada:
df.corr()['col_foco'].sort_values().drop('col_foco').plot.bar()
# interpolação via pandas com base no index: <https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html>
col_foco.interpolate()

In [ ]:
# registros duplicados
df.drop_duplicates(keep='first/last/False', inplace=True)  # subset=[col1, col2]  # para restringir a duplicidade

In [ ]:
drop_index = df.loc[df.col.isin(['str1', 'str2'])].index
df.drop(index = drop_index, inplace = True)

In [ ]:
{col:col for col in df.columns}
df.rename(columns={'col1':'new_col1'})

In [4]:
from math import log10  as log10

In [20]:
log10(2*10/10) / log10(2*1000/10)

0.13082402064781276

In [9]:
log10(10000)

4.0

In [3]:
np.log10(1e4)

4.0

In [36]:
np.random.seed(0)
cpfs = np.random.randint(0,10,(3,11))
cpfs

array([[5, 0, 3, 3, 7, 9, 3, 5, 2, 4, 7],
       [6, 8, 8, 1, 6, 7, 7, 8, 1, 5, 9],
       [8, 9, 4, 3, 0, 3, 5, 0, 2, 3, 8]])

In [39]:
cpfs[:, 9] = np.zeros((3), dtype= int)   # zera 2 ultimos digitos
cpfs[:, 10] = np.zeros((3), dtype= int)
cpfs

array([[5, 0, 3, 3, 7, 9, 3, 5, 2, 0, 0],
       [6, 8, 8, 1, 6, 7, 7, 8, 1, 0, 0],
       [8, 9, 4, 3, 0, 3, 5, 0, 2, 0, 0]])

In [41]:
w1 = np.zeros(11, int)
w1[0:9] = np.arange(10, 1, -1)
w1

array([10,  9,  8,  7,  6,  5,  4,  3,  2,  0,  0])

In [52]:
dv1 = np.dot(cpfs,w1) * 10 % 11 % 10
dv1

array([7, 2, 0], dtype=int32)

In [64]:
w2 = np.zeros(11, int)
w2[0:10] = np.arange(11, 1, -1)
w2

array([11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  0])

In [65]:
dv2 = np.dot(cpfs,w2) * 10 % 11 % 10
dv2

array([0, 1, 0], dtype=int32)

In [66]:
cpfs[:, 9] = dv1
cpfs[:, 10] = dv2
cpfs

array([[5, 0, 3, 3, 7, 9, 3, 5, 2, 7, 0],
       [6, 8, 8, 1, 6, 7, 7, 8, 1, 2, 1],
       [8, 9, 4, 3, 0, 3, 5, 0, 2, 0, 0]])

In [67]:
l_cpf = cpfs.astype(str).tolist()
l_cpf

[['5', '0', '3', '3', '7', '9', '3', '5', '2', '7', '0'],
 ['6', '8', '8', '1', '6', '7', '7', '8', '1', '2', '1'],
 ['8', '9', '4', '3', '0', '3', '5', '0', '2', '0', '0']]

In [68]:
[''.join(d) for d in l_cpf]

['50337935270', '68816778121', '89430350200']